# 语义解析：基于seq2seq的text2sparql

参考：[『NLP经典项目集』05：新年到，飞桨带你对对联](https://aistudio.baidu.com/aistudio/projectdetail/1321118)

SPARQL的英文全称为SPARQL Protocol and RDF Query Language，是为RDF开发的一种查询语言和数据获取协议，它是为W3C所开发的RDF数据模型所定义，但是可以用于任何可以用RDF来表示的信息资源。它于2008年1月15日正式成为一项W3C推荐标准，于2013年3月发布SPARQL1.1。

RDF的英语全称为Resource Description Framework，中文名称为资源描述框架。RDF是一种描述数据文件储存的数据模型，该数据模型通常描述由三个部分组成的事实，被称为三元组（triples）。三元组由主语（subject）、谓语（predicate）和宾语（object）组成，看上去很像一个简单的句子。


这里，我们将根据自然语言，自动写sparql。这是一个典型的序列到序列(sequence2sequence, seq2seq）建模的场景，编码器-解码器（Encoder-Decoder）框架是解决seq2seq问题的经典方法，它能够将一个任意长度的源序列转换成另一个任意长度的目标序列：编码阶段将整个源序列编码成一个向量，解码阶段通过最大化预测序列概率，从中解码出整个目标序列。编码和解码的过程通常都使用RNN实现。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/e9dde4be7d0142068c5c921a1ca6a227a49aad4a8751425faead42f0348f5e01" width="500" height="313" ></center>
<br><center>图1：encoder-decoder示意图</center></br>


这里的Encoder采用LSTM，Decoder采用带有attention机制的LSTM。 


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/a791fee76388423da867676d667b7d4c2fbe9fe9096843878c6513a40c96c86d" width="500" height="313" ></center>
<br><center>图2：带有attention机制的encoder-decoder示意图</center></br>

我们将以自然语言问句作为Encoder的输出，sparql作为Decoder的输入，训练模型。

AI Studio平台默认安装了Paddle和PaddleNLP，并定期更新版本。 如需手动更新Paddle，可参考[飞桨安装说明](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/conda/linux-conda.html)，安装相应环境下最新版飞桨框架。

使用如下命令确保安装最新版PaddleNLP：

In [1]:
# !pip install --upgrade paddlenlp -i https://pypi.org/simple

In [2]:
import paddlenlp
paddlenlp.__version__

'2.0.7'

In [3]:
import io
import os

from functools import partial

import numpy as np

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddlenlp.data import Vocab, Pad
from paddlenlp.metrics import Perplexity
from paddlenlp.datasets import load_dataset

# 数据部分

## 数据集介绍

由于数据集不好生成，所以我这边只列了7条进行测试。

数据如下：

```
煤炭行业有多少上市公司	select ( COUNT ( distinct ?result ) AS ?count ) where { ?n1 sct:hasChineseLabel "煤炭行业" . ?n1 rdfs: ?result . }
乐凯胶片的股票id是多少	select ?result where { ?n1 sct:hasChineseLabel "乐凯胶片" . ?n1 sct:ID ?result . }
太原重工属于哪个行业	select ?result where { ?n1 sct:hasChineseLabel ?result . ?n1 rdfs: ?n2 . ?n2 sct:hasChineseLabel "太原重工" . }
上市公司最多的行业是什么	select ?result where { { select ?n1 ( count ( ?n2 ) as ?subresult ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } ?n1 sct:hasChineseLabel ?result . } order by desc ( ?subresult ) limit 1
上市公司最少的5个行业是什么	select ?result where { { select ?n1 ( count ( ?n2 ) as ?subresult ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } ?n1 sct:hasChineseLabel ?result . } order by ?subresult limit 5
上市公司最多的前3个行业的公司数目各是多少	select ?result where { select ( count ( ?n2 ) as ?result ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } order by desc ( ?result ) limit 3
澳柯玛	select ?p ?result where { ?n1 sct:hasChineseLabel "澳柯玛" . ?n1 ?p ?result . }
```

## 加载数据集

读入数据文件，这里将所有的输入数据全部转为小写统一格式，

<br>

- 获取数据集可以调用`paddlenlp.datasets.load_dataset`。其中train_ds为训练集，用于模型训练；由于数据量有些少，我将测试集test_ds设置为与train_ds一样，用于评估算法的性能，但不会根据测试集上的表现再去调整模型或参数。

- 将生成的字典文件写入到data_dic.txt中保存，便于后台预测时读取调用。

<br>


## 将数据集文本转成id
使用jieba分词，然后将其映射成词向量，并拼接上开始和结束词。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/d6c36cfd88eb4d0d87884f6c9cd47e466c2b562411394606be6683af08733045" width="200" height="200" ></center>
<br><center>图3：token-to-id示意图</center></br>

In [4]:
import jieba
import json
from paddlenlp.datasets import load_dataset


pad_id = 2
bos_id = 1
eos_id = 2
data_path='data.txt'
data_dic_path = 'data_dic.txt'

def get_id(vocab_dict, token):
    return vocab_dict[token]

with open(data_path,'r',encoding='utf-8') as fp:
    lines=fp.readlines()
    vocabs=[]
    quers=[]
    asws=[]
    for line in lines:
        line = line.lower() #全部转为小写
        data=line.split('\t')
        quer=jieba.lcut(data[0].strip(), cut_all=False)
        asw=data[1].strip().split(' ')
        vocabs.extend(quer)
        vocabs.extend(asw)
        quers.append(quer)
        asws.append(asw)
    vocabs=list(set(vocabs))
    vocab_dict={v:k+3 for k,v in enumerate(vocabs)}
with open(data_dic_path,'w',encoding='utf-8') as fp:
    data_dic=json.dumps(vocab_dict,ensure_ascii=False)
    fp.write(data_dic)

def read(data_path,quers,asws,vocab_dict):
    for q,a in zip(quers,asws):
        quer = [bos_id]+[get_id(vocab_dict, v) for v in q]+[eos_id]
        asw = [bos_id]+[get_id(vocab_dict, v) for v in a]+[eos_id]
        #train_ds.append(([bos_id]+quer+[eos_id],[bos_id]+asw+[eos_id]))
        yield (quer, asw)
train_ds = load_dataset(read, data_path=data_path, quers=quers, asws=asws, vocab_dict=vocab_dict, lazy=False)
test_ds = train_ds
vocab_size = len(vocabs)+3
print(vocab_dict)
print(train_ds[0])
trg_idx2word = {k:v for v,k in vocab_dict.items()}

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.830 seconds.
Prefix dict has been built successfully.


{'"乐凯胶片"': 3, '煤炭行业': 4, 'group': 5, 'desc': 6, '个': 7, '(': 8, '.': 9, '澳柯玛': 10, 'distinct': 11, '乐凯胶片': 12, 'count': 13, '?count': 14, '公司': 15, '重工': 16, '上市公司': 17, '的': 18, 'rdfs:': 19, '属于': 20, '1': 21, ')': 22, 'where': 23, '太原': 24, '"太原重工"': 25, '3': 26, 'sct:id': 27, '哪个': 28, 'order': 29, '最多': 30, '{': 31, '最少': 32, 'as': 33, 'zg:sinafinance': 34, '?n1': 35, '}': 36, 'sct:haschineselabel': 37, 'by': 38, '各是': 39, 'limit': 40, '数目': 41, '多少': 42, 'select': 43, '是': 44, '有': 45, '什么': 46, '?subresult': 47, '?n2': 48, '?p': 49, '"澳柯玛"': 50, '行业': 51, 'id': 52, '5': 53, '?result': 54, '股票': 55, '前': 56, '"煤炭行业"': 57}
([1, 4, 45, 42, 17, 2], [1, 43, 8, 13, 8, 11, 54, 22, 33, 14, 22, 23, 31, 35, 37, 57, 9, 35, 19, 54, 9, 36, 2])


## 构造dataloder

使用`paddle.io.DataLoader`来创建训练和预测时所需要的`DataLoader`对象。

`paddle.io.DataLoader`返回一个迭代器，该迭代器根据`batch_sampler`指定的顺序迭代返回dataset数据。支持单进程或多进程加载数据，快！

<br>

接收如下重要参数：
-  `batch_sampler`：批采样器实例，用于在`paddle.io.DataLoader` 中迭代式获取mini-batch的样本下标数组，数组长度与 batch_size 一致。
-  `collate_fn`：指定如何将样本列表组合为mini-batch数据。传给它参数需要是一个`callable`对象，需要实现对组建的batch的处理逻辑，并返回每个batch的数据。在这里传入的是`prepare_input`函数，对产生的数据进行pad操作，并返回实际长度等。


PaddleNLP提供了许多NLP任务中，用于数据处理、组batch数据的相关API。

| API                             | 简介                                       |
| ------------------------------- | :----------------------------------------- |
| `paddlenlp.data.Stack`          | 堆叠N个具有相同shape的输入数据来构建一个batch |
| `paddlenlp.data.Pad`            | 将长度不同的多个句子padding到统一长度，取N个输入数据中的最大长度 |
| `paddlenlp.data.Tuple`          | 将多个batchify函数包装在一起 |

更多数据处理操作详见： [https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/data.md](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/data.md)

In [5]:
def create_data_loader(dataset):
    data_loader = paddle.io.DataLoader(
        dataset,
        batch_sampler=None,
        batch_size = batch_size,
        collate_fn=partial(prepare_input, pad_id=pad_id))
    return data_loader

def prepare_input(insts, pad_id):
    src, src_length = Pad(pad_val=pad_id, ret_length=True)([inst[0] for inst in insts])
    tgt, tgt_length = Pad(pad_val=pad_id, ret_length=True)([inst[1] for inst in insts])
    tgt_mask = (tgt[:, :-1] != pad_id).astype(paddle.get_default_dtype())
    #print(src, src_length, tgt[:, :-1], tgt[:, 1:, np.newaxis], tgt_mask)
    return src, src_length, tgt[:, :-1], tgt[:, 1:, np.newaxis], tgt_mask

In [6]:
device = "gpu" # or cpu
device = paddle.set_device(device)

batch_size = 7
num_layers = 2
dropout = 0.2
hidden_size =256
max_grad_norm = 5.0
learning_rate = 0.001
max_epoch = 300
model_path = './couplet_models'
log_freq = 2000

# Define dataloader
train_loader = create_data_loader(train_ds)
test_loader = create_data_loader(test_ds)

print(len(train_ds), len(train_loader), batch_size)
# 7 1 7  共1个batch

for i in train_loader:
    print (len(i))
    for ind, each in enumerate(i):
        print (ind, each.shape)
    break

7 1 7
5
0 [7, 14]
1 [7]
2 [7, 44]
3 [7, 44, 1]
4 [7, 44]


# 模型部分
下图是带有Attention的Seq2Seq模型结构。下面我们分别定义网络的每个部分，最后构建Seq2Seq主网络。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/8a9dda0434a14fb2a0837702e5f2f1096346810702aa4a6ab1fa7dafe548add6" width="600" height="600" ></center>
<br><center>图5：带有attention机制的encoder-decoder原理示意图</center></br>

## 定义Encoder

Encoder部分非常简单，可以直接利用PaddlePaddle2.0提供的RNN系列API的`nn.LSTM`。


1. `nn.Embedding`：该接口用于构建 Embedding 的一个可调用对象，根据输入的size (vocab_size, embedding_dim)自动构造一个二维embedding矩阵，用于table-lookup。查表过程如下：

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/54276195f4ce44b9ace89c5153300782a744a98343454410898fcfe81333f131" width="700" height="600" ></center>
<br><center>图5：token-to-id & 查表获取向量示意图</center></br>

2. `nn.LSTM`：提供序列，得到`encoder_output`和`encoder_state`。    
参数：   
- input_size (int) 输入的大小。
- hidden_size (int) - 隐藏状态大小。
- num_layers (int，可选) - 网络层数。默认为1。
- direction (str，可选) - 网络迭代方向，可设置为forward或bidirect（或bidirectional）。默认为forward。
- time_major (bool，可选) - 指定input的第一个维度是否是time steps。默认为False。
- dropout (float，可选) - dropout概率，指的是出第一层外每层输入时的dropout概率。默认为0。

[https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/layer/rnn/LSTM_cn.html](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/layer/rnn/LSTM_cn.html)


输出:

outputs (Tensor) - 输出，由前向和后向cell的输出拼接得到。如果time_major为True，则Tensor的形状为[time_steps,batch_size,num_directions * hidden_size]，如果time_major为False，则Tensor的形状为[batch_size,time_steps,num_directions * hidden_size]，当direction设置为bidirectional时，num_directions等于2，否则等于1。

final_states (tuple) - 最终状态,一个包含h和c的元组。形状为[num_lauers * num_directions, batch_size, hidden_size],当direction设置为bidirectional时，num_directions等于2，否则等于1。

In [7]:
class Seq2SeqEncoder(nn.Layer):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers):
        super(Seq2SeqEncoder, self).__init__()
        self.embedder = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=0.2 if num_layers > 1 else 0.)

    def forward(self, sequence, sequence_length):
        inputs = self.embedder(sequence)
        encoder_output, encoder_state = self.lstm(
            inputs, sequence_length=sequence_length)
        
        # encoder_output [128, 18, 256]  [batch_size, time_steps, hidden_size]
        # encoder_state (tuple) - 最终状态,一个包含h和c的元组。 [2, 128, 256] [2, 128, 256] [num_layers * num_directions, batch_size, hidden_size]
        return encoder_output, encoder_state

## 定义Decoder

###  定义AttentionLayer
1. `nn.Linear`线性变换层传入2个参数   
- in_features (int) – 线性变换层输入单元的数目。
- out_features (int) – 线性变换层输出单元的数目。

![](https://ai-studio-static-online.cdn.bcebos.com/4ce5851e5eab41af9733962d1782669dbef31a74540e468ab05b730ab2bb4ffc)

2. `paddle.matmul`用于计算两个Tensor的乘积，遵循完整的广播规则，关于广播规则，请参考[广播 (broadcasting)](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/01_paddle2.0_introduction/basic_concept/broadcasting_cn.html#cn-user-guide-broadcasting) 。 并且其行为与 numpy.matmul 一致。  
- x (Tensor) : 输入变量，类型为 Tensor，数据类型为float32， float64。
- y (Tensor) : 输入变量，类型为 Tensor，数据类型为float32， float64。
- transpose_x (bool，可选) : 相乘前是否转置 x，默认值为False。
- transpose_y (bool，可选) : 相乘前是否转置 y，默认值为False。

<br>

3. `paddle.unsqueeze`用于向输入Tensor的Shape中一个或多个位置（axis）插入尺寸为1的维度

4. `paddle.add`逐元素相加算子，输入 x 与输入 y 逐元素相加，并将各个位置的输出元素保存到返回结果中。

输入 x 与输入 y 必须可以广播为相同形状。

In [8]:
class AttentionLayer(nn.Layer):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.input_proj = nn.Linear(hidden_size, hidden_size)
        self.output_proj = nn.Linear(hidden_size + hidden_size, hidden_size)

    def forward(self, hidden, encoder_output, encoder_padding_mask):
        encoder_output = self.input_proj(encoder_output)
        attn_scores = paddle.matmul(
            paddle.unsqueeze(hidden, [1]), encoder_output, transpose_y=True)
        # print('attention score', attn_scores.shape) #[128, 1, 18]

        if encoder_padding_mask is not None:
            attn_scores = paddle.add(attn_scores, encoder_padding_mask)

        attn_scores = F.softmax(attn_scores)
        attn_out = paddle.squeeze(
            paddle.matmul(attn_scores, encoder_output), [1])
        # print('1 attn_out', attn_out.shape) #[128, 256]

        attn_out = paddle.concat([attn_out, hidden], 1)
        # print('2 attn_out', attn_out.shape) #[128, 512]

        attn_out = self.output_proj(attn_out)
        # print('3 attn_out', attn_out.shape) #[128, 256]
        return attn_out

### 定义Seq2SeqDecoderCell
由于Decoder部分是带有attention的LSTM，我们不能复用`nn.LSTM`，所以需要定义`Seq2SeqDecoderCell`

1. `nn.LayerList` 用于保存子层列表，它包含的子层将被正确地注册和添加。列表中的子层可以像常规python列表一样被索引。这里添加了num_layers=2层lstm。

In [9]:
class Seq2SeqDecoderCell(nn.RNNCellBase):
    def __init__(self, num_layers, input_size, hidden_size):
        super(Seq2SeqDecoderCell, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.lstm_cells = nn.LayerList([
            nn.LSTMCell(
                input_size=input_size + hidden_size if i == 0 else hidden_size,
                hidden_size=hidden_size) for i in range(num_layers)
        ])

        self.attention_layer = AttentionLayer(hidden_size)
    
    def forward(self,
                step_input,
                states,
                encoder_output,
                encoder_padding_mask=None):
        lstm_states, input_feed = states
        new_lstm_states = []
        step_input = paddle.concat([step_input, input_feed], 1)
        for i, lstm_cell in enumerate(self.lstm_cells):
            out, new_lstm_state = lstm_cell(step_input, lstm_states[i])
            step_input = self.dropout(out)
            new_lstm_states.append(new_lstm_state)
        out = self.attention_layer(step_input, encoder_output,
                                   encoder_padding_mask)
        return out, [new_lstm_states, out]

### 定义Seq2SeqDecoder
有了`Seq2SeqDecoderCell`，就可以构建`Seq2SeqDecoder`了

<br>

1. `paddle.nn.RNN` 该OP是循环神经网络（RNN）的封装，将输入的Cell封装为一个循环神经网络。它能够重复执行 cell.forward() 直到遍历完input中的所有Tensor。
- cell (RNNCellBase) - RNNCellBase类的一个实例。

In [10]:
class Seq2SeqDecoder(nn.Layer):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers):
        super(Seq2SeqDecoder, self).__init__()
        self.embedder = nn.Embedding(vocab_size, embed_dim)
        self.lstm_attention = nn.RNN(
            Seq2SeqDecoderCell(num_layers, embed_dim, hidden_size))
        self.output_layer = nn.Linear(hidden_size, vocab_size)

    def forward(self, trg, decoder_initial_states, encoder_output,
                encoder_padding_mask):
        inputs = self.embedder(trg)

        decoder_output, _ = self.lstm_attention(
            inputs,
            initial_states=decoder_initial_states,
            encoder_output=encoder_output,
            encoder_padding_mask=encoder_padding_mask)
        predict = self.output_layer(decoder_output)

        return predict

## 构建主网络Seq2SeqAttnModel
Encoder和Decoder定义好之后，网络就可以构建起来了

In [11]:
class Seq2SeqAttnModel(nn.Layer):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers,
                 eos_id=1):
        super(Seq2SeqAttnModel, self).__init__()
        self.hidden_size = hidden_size
        self.eos_id = eos_id
        self.num_layers = num_layers
        self.INF = 1e9
        self.encoder = Seq2SeqEncoder(vocab_size, embed_dim, hidden_size,
                                      num_layers)
        self.decoder = Seq2SeqDecoder(vocab_size, embed_dim, hidden_size,
                                      num_layers)

    def forward(self, src, src_length, trg):
        # encoder_output 各时刻的输出h
        # encoder_final_state 最后时刻的输出h，和记忆信号c
        encoder_output, encoder_final_state = self.encoder(src, src_length)
        # print('encoder_output shape', encoder_output.shape)  #  [128, 18, 256]  [batch_size,time_steps,hidden_size]
        # print('encoder_final_states shape', encoder_final_state[0].shape, encoder_final_state[1].shape) #[2, 128, 256] [2, 128, 256] [num_lauers * num_directions, batch_size, hidden_size]

        # Transfer shape of encoder_final_states to [num_layers, 2, batch_size, hidden_size]？？？
        encoder_final_states = [
            (encoder_final_state[0][i], encoder_final_state[1][i])
            for i in range(self.num_layers)
        ]
        # print('encoder_final_states shape', encoder_final_states[0][0].shape, encoder_final_states[0][1].shape) #[128, 256] [128, 256]


        # Construct decoder initial states: use input_feed and the shape is
        # [[h,c] * num_layers, input_feed], consistent with Seq2SeqDecoderCell.states
        decoder_initial_states = [
            encoder_final_states,
            self.decoder.lstm_attention.cell.get_initial_states(
                batch_ref=encoder_output, shape=[self.hidden_size])
        ]

        # Build attention mask to avoid paying attention on padddings
        src_mask = (src != self.eos_id).astype(paddle.get_default_dtype())
        # print ('src_mask shape', src_mask.shape)  #[128, 18]
        # print(src_mask[0, :])

        encoder_padding_mask = (src_mask - 1.0) * self.INF
        # print ('encoder_padding_mask', encoder_padding_mask.shape)  #[128, 18]
        # print(encoder_padding_mask[0, :])

        encoder_padding_mask = paddle.unsqueeze(encoder_padding_mask, [1])
        # print('encoder_padding_mask', encoder_padding_mask.shape)  #[128, 1, 18]

        predict = self.decoder(trg, decoder_initial_states, encoder_output,
                               encoder_padding_mask)
        # print('predict', predict.shape)   #[128, 17, 7931]

        return predict


## 定义损失函数
这里使用的是交叉熵损失函数，我们需要将padding位置的loss置为0，因此需要在损失函数中引入`trg_mask`参数，由于PaddlePaddle框架提供的`paddle.nn.CrossEntropyLoss`不能接受`trg_mask`参数，因此在这里需要重新定义：

In [12]:
class CrossEntropyCriterion(nn.Layer):
    def __init__(self):
        super(CrossEntropyCriterion, self).__init__()

    def forward(self, predict, label, trg_mask):
        cost = F.softmax_with_cross_entropy(
            logits=predict, label=label, soft_label=False)
        cost = paddle.squeeze(cost, axis=[2])
        masked_cost = cost * trg_mask
        batch_mean_cost = paddle.mean(masked_cost, axis=[0])
        seq_cost = paddle.sum(batch_mean_cost)

        return seq_cost

# 执行过程
## 训练过程
使用高层API执行训练，需要调用`prepare`和`fit`函数。

在`prepare`函数中，配置优化器、损失函数，以及评价指标。其中评价指标使用的是PaddleNLP提供的困惑度计算API `paddlenlp.metrics.Perplexity`。

如果你安装了VisualDL，可以在fit中添加一个callbacks参数使用VisualDL观测你的训练过程，如下：

``` python
model.fit(train_data=train_loader,
            epochs=max_epoch,
            eval_freq=1,
            save_freq=1,
            save_dir=model_path,
            log_freq=log_freq,
            callbacks=[paddle.callbacks.VisualDL('./log')])
```

在这里，由于对联生成任务没有明确的评价指标，因此，可以在保存的多个模型中，通过人工评判生成结果选择最好的模型。

本项目中，为了便于演示，已经将训练好的模型参数载入模型，并省略了训练过程。读者自己实验的时候，可以尝试自行修改超参数，调用下面被注释掉的`fit`函数，重新进行训练。

如果读者想要在更短的时间内得到效果不错的模型，可以使用预训练模型技术，例如[《预训练模型ERNIE-GEN自动写诗》](https://aistudio.baidu.com/aistudio/projectdetail/1339888)项目为大家展示了如何利用预训练的生成模型进行训练。

In [13]:
model = paddle.Model(
    Seq2SeqAttnModel(vocab_size, hidden_size, hidden_size,
                        num_layers, pad_id))

optimizer = paddle.optimizer.Adam(
    learning_rate=learning_rate, parameters=model.parameters())
ppl_metric = Perplexity()
model.prepare(optimizer, CrossEntropyCriterion(), ppl_metric)

model.fit(train_data=train_loader,
            epochs=max_epoch,
            eval_freq=10,
            save_freq=10,
            save_dir=model_path,
            log_freq=log_freq)

W0624 13:58:27.441231   183 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0624 13:58:27.445106   183 device_context.cc:465] device: 0, cuDNN Version: 7.6.


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/300
step 1/1 - loss: 110.5153 - Perplexity: 57.4147 - 171ms/step
save checkpoint at /home/aistudio/couplet_models/0
Epoch 2/300
step 1/1 - loss: 107.7036 - Perplexity: 51.7929 - 149ms/step
Epoch 3/300
step 1/1 - loss: 104.2128 - Perplexity: 45.5730 - 140ms/step
Epoch 4/300
step 1/1 - loss: 98.4705 - Perplexity: 36.9241 - 135ms/step
Epoch 5/300
step 1/1 - loss: 96.6868 - Perplexity: 34.5876 - 124ms/step
Epoch 6/300
step 1/1 - loss: 90.1235 - Perplexity: 27.1928 - 127ms/step
Epoch 7/300
step 1/1 - loss: 89.0746 - Perplexity: 26.1674 - 127ms/step
Epoch 8/300
step 1/1 - loss: 87.7988 - Perplexity: 24.9720 - 134ms/step
Epoch 9/300
step 1/1 - loss: 86.2980 - Perplexity: 23.6356 - 127ms/step
Epoch 10/300
step 1/1 - loss: 84.0344 - Perplexity: 21.7540 - 132ms/step
Epoch 11/300
step 1/1 - loss: 81.8703 - Perplexity: 20.0952 - 140ms/step
save checkpoint at /home/aistudio/couplet_

## 模型预测

### 定义预测网络Seq2SeqAttnInferModel
预测网络继承上面的主网络`Seq2SeqAttnModel`，定义子类`Seq2SeqAttnInferModel`


In [14]:
class Seq2SeqAttnInferModel(Seq2SeqAttnModel):
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 hidden_size,
                 num_layers,
                 bos_id=0,
                 eos_id=1,
                 beam_size=4,
                 max_out_len=256):
        self.bos_id = bos_id
        self.beam_size = beam_size
        self.max_out_len = max_out_len
        self.num_layers = num_layers
        super(Seq2SeqAttnInferModel, self).__init__(
            vocab_size, embed_dim, hidden_size, num_layers, eos_id)

        # Dynamic decoder for inference
        self.beam_search_decoder = nn.BeamSearchDecoder(
            self.decoder.lstm_attention.cell,
            start_token=bos_id,
            end_token=eos_id,
            beam_size=beam_size,
            embedding_fn=self.decoder.embedder,
            output_fn=self.decoder.output_layer)

    def forward(self, src, src_length):
        encoder_output, encoder_final_state = self.encoder(src, src_length)

        encoder_final_state = [
            (encoder_final_state[0][i], encoder_final_state[1][i])
            for i in range(self.num_layers)
        ]

        # Initial decoder initial states
        decoder_initial_states = [
            encoder_final_state,
            self.decoder.lstm_attention.cell.get_initial_states(
                batch_ref=encoder_output, shape=[self.hidden_size])
        ]
        # Build attention mask to avoid paying attention on paddings
        src_mask = (src != self.eos_id).astype(paddle.get_default_dtype())

        encoder_padding_mask = (src_mask - 1.0) * self.INF
        encoder_padding_mask = paddle.unsqueeze(encoder_padding_mask, [1])

        # Tile the batch dimension with beam_size
        encoder_output = nn.BeamSearchDecoder.tile_beam_merge_with_batch(
            encoder_output, self.beam_size)
        encoder_padding_mask = nn.BeamSearchDecoder.tile_beam_merge_with_batch(
            encoder_padding_mask, self.beam_size)

        # Dynamic decoding with beam search
        seq_output, _ = nn.dynamic_decode(
            decoder=self.beam_search_decoder,
            inits=decoder_initial_states,
            max_step_num=self.max_out_len,
            encoder_output=encoder_output,
            encoder_padding_mask=encoder_padding_mask)
        return seq_output

## 解码部分
接下来对我们的任务选择beam search解码方式，可以指定beam_size为10。

In [15]:
def post_process_seq(seq, bos_idx, eos_idx, output_bos=False, output_eos=False):
    """
    Post-process the decoded sequence.
    """
    eos_pos = len(seq) - 1
    for i, idx in enumerate(seq):
        if idx == eos_idx:
            eos_pos = i
            break
    seq = [
        idx for idx in seq[:eos_pos + 1]
        if (output_bos or idx != bos_idx) and (output_eos or idx != eos_idx)
    ]
    return seq

In [16]:
beam_size = 10
model = paddle.Model(
    Seq2SeqAttnInferModel(
        vocab_size,
        hidden_size,
        hidden_size,
        num_layers,
        bos_id=bos_id,
        eos_id=eos_id,
        beam_size=beam_size,
        max_out_len=256))

model.prepare()

在预测之前，我们需要将训练好的模型参数load进预测网络，之后我们就可以根据自然语言文具，生成对应的sparql啦！

In [17]:
model.load('couplet_models/final')

In [18]:
idx = 0
trg_idx2word[1]='<start>'
trg_idx2word[2]='<end>'
for data in test_loader():
    inputs = data[:2]
    finished_seq = model.predict_batch(inputs=list(inputs))[0]
    finished_seq = finished_seq[:, :, np.newaxis] if len(
        finished_seq.shape) == 2 else finished_seq
    finished_seq = np.transpose(finished_seq, [0, 2, 1])
    for ins in finished_seq:
        for beam in ins:
            id_list = post_process_seq(beam, bos_id, eos_id)
            word_list_f = [trg_idx2word[id] for id in test_ds[idx][0]][1:-1]
            word_list_s = [trg_idx2word[id] for id in id_list]
            sequence = "问: "+"".join(word_list_f)+"\tsparql: "+" ".join(word_list_s).replace('sct:haschineselabel','sct:hasChineseLabel').replace('sct:id','sct:ID').replace('zg:sinafinance','zg:SinaFinance') + "\n"
            print(sequence)
            idx += 1
            break

问: 煤炭行业有多少上市公司	sparql: select ( count ( distinct ?result ) as ?count ) where { ?n1 sct:hasChineseLabel "煤炭行业" . ?n1 rdfs: ?result . }

问: 乐凯胶片的股票id是多少	sparql: select ?result where { ?n1 sct:hasChineseLabel "乐凯胶片" . ?n1 sct:ID ?result . }

问: 太原重工属于哪个行业	sparql: select ?result where { ?n1 sct:hasChineseLabel ?result . ?n1 rdfs: ?n2 . ?n2 sct:hasChineseLabel "太原重工" . }

问: 上市公司最多的行业是什么	sparql: select ?result where { { select ?n1 ( count ( ?n2 ) as ?subresult ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } ?n1 sct:hasChineseLabel ?result . } order by desc ( ?subresult ) limit 1

问: 上市公司最少的5个行业是什么	sparql: select ?result where { { select ?n1 ( count ( ?n2 ) as ?subresult ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } ?n1 sct:hasChineseLabel ?result . } order by desc ( ?subresult ) limit 1

问: 上市公司最多的前3个行业的公司数目各是多少	sparql: select ?result where { select ( count ( ?n2 ) as ?result ) where { zg:SinaFinance rdfs: ?n1 . ?n1 rdfs: ?n2 . } group by ?n1 } 